# Data Cleaning

I've collected data from the top 10,000 popular movies ranging from summer 2010 to summer 2020 on IMDB using web scraping and combined this with budget and revenue data for these movies collected from TMDB using their API. 

Next, I'll get a sense of what the data looks like.

## Import Libraries and Data

In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle("./movie_data.pkl")

## Explore Data

First, I want to make sure the shape of the dataframe is what I would expect. I collected 10,000 movies and for each I grabbed the following information.
* `title`
* `mpaa_rating`
* `runtime`
* `genre`
* `star_rating` - This is the rating the movie has on IMDB out of 10.
* `budget`
* `revenue`

As such, I expect my dataframe to have 10,000 rows and 7 columns.

In [3]:
df.shape

(10000, 7)

Great. Now I want to see what this dataframe looks like.

In [4]:
df.head()

,title,mpaa_rating,runtime,genre,star_rating,budget,revenue
0,The Outpost,R,123 min,"\nAction, Drama, History",6.7,0.0,0.0
1,The Gentlemen,R,113 min,"\nAction, Comedy, Crime",7.9,22000000.0,114996853.0
2,Murder on the Orient Express,PG-13,114 min,"\nCrime, Drama, Mystery",6.5,55000000.0,351839303.0
3,365 Days,TV-MA,114 min,"\nDrama, Romance",3.3,0.0,9458590.0
4,Mulan,PG-13,115 min,"\nAction, Adventure, Drama",5.4,200000000.0,57000000.0


That looks correct. Next, I want to look at my columns more in depth.

I'll start by looking at the value count of my categorical column `mpaa_rating`.

In [22]:
df['mpaa_rating'].value_counts(dropna=False)

NaN          2754
R            2425
Not Rated    2029
PG-13        1128
TV-MA         489
PG            457
Unrated       306
TV-14         270
TV-PG          70
G              35
TV-G           16
TV-Y7           7
NC-17           5
M               5
TV-Y            2
MA-17           1
18              1
Name: mpaa_rating, dtype: int64

In [ ]:
#TODO: remove na, combine not rated and unrated, research what else can be combined.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
title          10000 non-null object
mpaa_rating    7246 non-null object
runtime        9909 non-null object
genre          9999 non-null object
star_rating    9987 non-null object
budget         9925 non-null float64
revenue        9925 non-null float64
dtypes: float64(2), object(5)
memory usage: 547.0+ KB


It looks like two of my columns are not the right data type; `star_rating` should be a float, and `runtime` should be an integer. The first will be an easy fix, but since `runtime` contains both numbers and text, I want to turn it into a string, strip the text I don't want, and then turn the column into type int. Before I can do this, I'll have to check for null values.

## Data Cleaning

From the table, I see that some budget and revenue numbers are listed as 0. When analyzing movie budgets and revenue, I only want to analyze movies which have these data points. I'll create a new dataframe `df_short` which I can use when analyzing the finances of the movies. 

To start off, I'll see how many 0 values I have for these.

In [6]:
df.isin([0]).sum()

title             0
mpaa_rating       0
runtime           0
genre             0
star_rating       0
budget         6870
revenue        7047
dtype: int64

In [17]:
df_short = df[df['revenue'] != 0] 
df_short = df_short[df_short['budget'] != 0]

In [19]:
df_short.shape

(2140, 7)

Looks like that leaves me with 2,140 movie details to use for analyzing popular movie finances.

Next, I'll look at null values.

In [7]:
df.isna().sum()

title             0
mpaa_rating    2754
runtime          91
genre             1
star_rating      13
budget           75
revenue          75
dtype: int64